In [1]:
import datatable as dt
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
rename = {}
idx = 0
for i in os.listdir('.'):
    if i[:3]=='GSM' and i[-3:]=='bed' and i.find('Blood')!=-1:
        rename[idx] = {}
        rename[idx]['ID'] = i.split('_')[0]
        rename[idx]["type"] = i.split('Blood-')[1].split('-')[0]
        idx+=1

In [3]:
type = {}
idx = 0
for i in pd.DataFrame(rename).T['type'].unique():
    type[i] = idx
    idx +=1 
type

{'T': 0, 'NK': 1, 'Monocytes': 2, 'Granulocytes': 3, 'B': 4}

In [ ]:
df = pd.read_table("./Blood.metilene.chr2x.input", header=None, skiprows=3)
df.columns = 'chrom	pos'.split('\t')+list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])
df

In [ ]:
# 3261322	3261610
cm = sns.clustermap(1*df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)].fillna(100)\
               [list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])].T,\
               cmap='mako_r', figsize=[6,10], method='ward', dendrogram_ratio=0.01, xticklabels=False, col_cluster=False, row_cluster=False)
cm.cax.set_visible(False)
# cm.fig.suptitle('pub') 

In [ ]:
df.dropna().to_csv('./Blood.metilene.chr2x.noNA.input',sep='\t',index=False, float_format='%.3f')

In [ ]:
# 3261322	3261610
cm = sns.clustermap(1*df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)].dropna()\
               [list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])].T,\
               cmap='coolwarm', figsize=[6,10], method='ward', dendrogram_ratio=0.01, xticklabels=False, col_cluster=False, row_cluster=False)
cm.cax.set_visible(False)
# cm.fig.suptitle('pub') 

In [ ]:
means = {}
for i in type.values():
    means[str(i)] = df[df.columns[df.columns.str.contains(str(i)+'_')]].T.mean()

for i in list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID']):
    df[i] = df[i].fillna(means[i.split('_')[0]])

df = df.fillna(0.5)
df.to_csv('./Blood.metilene.chr2x.fillNA.input',sep='\t',index=False, float_format='%.3f')
df

In [ ]:
# 3261322	3261610
cm = sns.clustermap(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)].dropna()\
               [list(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'])].T,\
               cmap='coolwarm', figsize=[6,10], method='ward', dendrogram_ratio=0.01, xticklabels=False, col_cluster=False, row_cluster=False)
cm.cax.set_visible(False)
# cm.fig.suptitle('pub') 

In [ ]:
sns.color_palette("Paired")

In [ ]:
param_m = '.m5'
param_NA = 'noNA'

DMRs = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', skiprows=90)
id_convert = {}
Predictions_anno = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', encoding='iso-8859-1', sep='@', header=None).head(90)
for i in Predictions_anno.index:
    id_convert[int(Predictions_anno.loc[i][0].split(':')[0].split('Combination ')[1])] = \
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[0]+'|'+\
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[1][:-1]
    
DMRs = DMRs.loc[DMRs['chr'] =='chr2']
inDMRs = {}
inDMRs2 = {}
for i in DMRs.index:
    for j in range(int(DMRs['start'][i])+1, int(DMRs['stop'][i]+1)):
        inDMRs[j] = DMRs['sig.comparison'][i]
        inDMRs2[j] = DMRs['p'][i]

df = pd.read_table('./Blood.metilene.chr2x.'+param_NA+'.input')

DMR_ids = sorted(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).dropna().unique())

cDMRs = {}
for i in range(len(DMR_ids)):
    print(i, DMR_ids[i], id_convert[DMR_ids[i]])
    cDMRs[DMR_ids[i]] = sns.color_palette("Paired")[i]
    

cm = sns.clustermap(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]\
               [sorted(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'], reverse=True)].T,\
               col_colors=df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).map(cDMRs),\
               cmap='coolwarm', figsize=[6,10], method='ward', dendrogram_ratio=0.01, xticklabels=False, col_cluster=False, row_cluster=False)
cm.cax.set_visible(False)

In [ ]:
param_m = ''
param_NA = 'noNA'

DMRs = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', skiprows=90)
id_convert = {}
Predictions_anno = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', encoding='iso-8859-1', sep='@', header=None).head(90)
for i in Predictions_anno.index:
    id_convert[int(Predictions_anno.loc[i][0].split(':')[0].split('Combination ')[1])] = \
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[0]+'|'+\
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[1][:-1]
    
DMRs = DMRs.loc[DMRs['chr'] =='chr2']
inDMRs = {}
inDMRs2 = {}
for i in DMRs.index:
    for j in range(int(DMRs['start'][i])+1, int(DMRs['stop'][i]+1)):
        inDMRs[j] = DMRs['sig.comparison'][i]
        inDMRs2[j] = DMRs['p'][i]

df = pd.read_table('./Blood.metilene.chr2x.'+param_NA+'.input')

DMR_ids = sorted(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).dropna().unique())

cDMRs = {}
for i in range(len(DMR_ids)):
    print(i, DMR_ids[i], id_convert[DMR_ids[i]])
    cDMRs[DMR_ids[i]] = sns.color_palette("Paired")[i]
    

cm = sns.clustermap(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]\
               [sorted(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'], reverse=True)].T,\
               col_colors=df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).map(cDMRs),\
               cmap='coolwarm', figsize=[6,10], method='ward', dendrogram_ratio=0.01, xticklabels=False, col_cluster=False, row_cluster=False)
cm.cax.set_visible(False)

In [ ]:
param_m = '.m5'
param_NA = 'fillNA'

DMRs = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', skiprows=90)
id_convert = {}
Predictions_anno = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', encoding='iso-8859-1', sep='@', header=None).head(90)
for i in Predictions_anno.index:
    id_convert[int(Predictions_anno.loc[i][0].split(':')[0].split('Combination ')[1])] = \
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[0]+'|'+\
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[1][:-1]
    
DMRs = DMRs.loc[DMRs['chr'] =='chr2']
inDMRs = {}
inDMRs2 = {}
for i in DMRs.index:
    for j in range(int(DMRs['start'][i])+1, int(DMRs['stop'][i]+1)):
        inDMRs[j] = DMRs['sig.comparison'][i]
        inDMRs2[j] = DMRs['p'][i]

df = pd.read_table('./Blood.metilene.chr2x.'+param_NA+'.input')

DMR_ids = sorted(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).dropna().unique())

cDMRs = {}
for i in range(len(DMR_ids)):
    print(i, DMR_ids[i], id_convert[DMR_ids[i]])
    cDMRs[DMR_ids[i]] = sns.color_palette("Paired")[i]
    

cm = sns.clustermap(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]\
               [sorted(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'], reverse=True)].T,\
               col_colors=df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).map(cDMRs),\
               cmap='coolwarm', figsize=[6,10], method='ward', dendrogram_ratio=0.01, xticklabels=False, col_cluster=False, row_cluster=False)
cm.cax.set_visible(False)

In [ ]:
param_m = ''
param_NA = 'fillNA'

DMRs = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', skiprows=90)
id_convert = {}
Predictions_anno = pd.read_table('../metilene_output/Blood.metilene'+param_m+'.chr2x.'+param_NA+'.output', encoding='iso-8859-1', sep='@', header=None).head(90)
for i in Predictions_anno.index:
    id_convert[int(Predictions_anno.loc[i][0].split(':')[0].split('Combination ')[1])] = \
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[0]+'|'+\
    Predictions_anno.loc[i][0].split('Group A subgroups: ')[1].split(',	Group B subgroups: ')[1][:-1]
    
DMRs = DMRs.loc[DMRs['chr'] =='chr2']
inDMRs = {}
inDMRs2 = {}
for i in DMRs.index:
    for j in range(int(DMRs['start'][i])+1, int(DMRs['stop'][i]+1)):
        inDMRs[j] = DMRs['sig.comparison'][i]
        inDMRs2[j] = DMRs['p'][i]

df = pd.read_table('./Blood.metilene.chr2x.'+param_NA+'.input')

DMR_ids = sorted(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).dropna().unique())

cDMRs = {}
for i in range(len(DMR_ids)):
    print(i, DMR_ids[i], id_convert[DMR_ids[i]])
    cDMRs[DMR_ids[i]] = sns.color_palette("Paired")[i]
    

cm = sns.clustermap(df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]\
               [sorted(pd.DataFrame(rename).T['type'].map(type).astype(str)+'_'+pd.DataFrame(rename).T['ID'], reverse=True)].T,\
               col_colors=df.loc[(df['chrom']=='chr2')&(df['pos']>= 3255001)&(df['pos']<=3273001)]['pos'].map(inDMRs).map(cDMRs),\
               cmap='coolwarm', figsize=[6,10], method='ward', dendrogram_ratio=0.01, xticklabels=False, col_cluster=False, row_cluster=False)
cm.cax.set_visible(False)